In [7]:
import pandas as pd
from tqdm import tqdm

## Sobre o dataset:
---
Identificação geográfica

CBSA → Core Based Statistical Area code
Código único da área estatística (MSA ou µSA).

MDIV → Metropolitan Division code
Subdivisão de algumas áreas metropolitanas muito grandes (ex.: Los Angeles, New York).

Se vazio, significa que a área não tem divisão.

STCOU → State + County FIPS code (5 dígitos)
Concatenado: 2 dígitos do estado (FIPS code) + 3 dígitos do condado.

NAME → Nome do município/condado/região (ex.: Los Angeles County, California).

LSAD → Legal/Statistical Area Description
Tipo da unidade (ex.: County, City, Metropolitan Statistical Area, Micropolitan Statistical Area, Metropolitan Division).

---
População base e estimada

ESTIMATESBASE2020 → População base do Censo de 2020 (ajustada como referência para projeções).

POPESTIMATE2020 ... POPESTIMATE2023 (ou 2024) → Estimativas de população anual para aquele condado/área em 1º de julho de cada ano.

---
Componentes de variação populacional

Essas vêm do modelo de componentes demográficos que o Census usa:

BIRTHSYYYY → Número de nascimentos no ano YYYY.

DEATHSYYYY → Número de óbitos no ano YYYY.

NATURALCHYYYY → Crescimento natural (nascimentos – mortes).

NETMIGYYYY → Migração líquida (entradas – saídas), incluindo doméstica e internacional.

RESIDUALYYYY → Ajuste estatístico que fecha a conta (quando os componentes não batem exatamente com a estimativa populacional).

---
As primeiras colunas (CBSA, MDIV, STCOU, NAME, LSAD) → identificação geográfica.

ESTIMATESBASE e POPESTIMATE → populações.

NETMIG, RESIDUAL, NATURALCH, etc. → componentes que explicam a variação da população ano a ano.

In [5]:
df = pd.read_csv('cbsa-est2024-alldata.csv', encoding="latin1")
df.head()

,CBSA,MDIV,STCOU,NAME,LSAD,ESTIMATESBASE2020,POPESTIMATE2020,POPESTIMATE2021,POPESTIMATE2022,POPESTIMATE2023,...,NETMIG2020,NETMIG2021,NETMIG2022,NETMIG2023,NETMIG2024,RESIDUAL2020,RESIDUAL2021,RESIDUAL2022,RESIDUAL2023,RESIDUAL2024
0,10180,NaN,NaN,"Abilene, TX",Metropolitan Statistical Area,176611,176926,177918,180072,182838,...,198,1161,2111,2457,1347,51,63,-26,3,-1
1,10180,NaN,48059.0,"Callahan County, TX",County or equivalent,13710,13753,14106,14275,14455,...,71,477,244,202,197,-6,-19,0,-2,-3
2,10180,NaN,48253.0,"Jones County, TX",County or equivalent,19663,19680,19903,20316,20765,...,26,373,464,489,163,6,-9,27,13,0
3,10180,NaN,48441.0,"Taylor County, TX",County or equivalent,143238,143493,143909,145481,147618,...,101,311,1403,1766,987,51,91,-53,-8,2
4,10420,NaN,NaN,"Akron, OH",Metropolitan Statistical Area,702211,701689,696602,698506,700094,...,-250,-3363,4405,3128,3399,196,216,-186,-148,3


In [13]:
df['STATE'] = ''

states = set()

for index, row in tqdm(df.iterrows(), total=df.shape[0]):
    name = row['NAME']
    state = name.split(', ')[-1].split('-')[0]
    df.at[index, 'STATE'] = state
    states.add(state)

print(f'Número de estados únicos: {len(states)}')

df.head()

100%|██████████| 2806/2806 [00:00<00:00, 14617.80it/s]

Número de estados únicos: 51


,CBSA,MDIV,STCOU,NAME,LSAD,ESTIMATESBASE2020,POPESTIMATE2020,POPESTIMATE2021,POPESTIMATE2022,POPESTIMATE2023,...,NETMIG2021,NETMIG2022,NETMIG2023,NETMIG2024,RESIDUAL2020,RESIDUAL2021,RESIDUAL2022,RESIDUAL2023,RESIDUAL2024,STATE
0,10180,NaN,NaN,"Abilene, TX",Metropolitan Statistical Area,176611,176926,177918,180072,182838,...,1161,2111,2457,1347,51,63,-26,3,-1,TX
1,10180,NaN,48059.0,"Callahan County, TX",County or equivalent,13710,13753,14106,14275,14455,...,477,244,202,197,-6,-19,0,-2,-3,TX
2,10180,NaN,48253.0,"Jones County, TX",County or equivalent,19663,19680,19903,20316,20765,...,373,464,489,163,6,-9,27,13,0,TX
3,10180,NaN,48441.0,"Taylor County, TX",County or equivalent,143238,143493,143909,145481,147618,...,311,1403,1766,987,51,91,-53,-8,2,TX
4,10420,NaN,NaN,"Akron, OH",Metropolitan Statistical Area,702211,701689,696602,698506,700094,...,-3363,4405,3128,3399,196,216,-186,-148,3,OH


In [14]:
cols_numericas = df.drop(columns=["CBSA", "MDIV", "STCOU", "NAME", "LSAD", "STATE"]).columns

df_estado = df.groupby("STATE")[cols_numericas].sum().reset_index()

print(df_estado.head())

  STATE  ESTIMATESBASE2020  POPESTIMATE2020  POPESTIMATE2021  POPESTIMATE2022  \
0    AK            1080382          1079782          1082544          1084470   
1    AL            9139847          9157448          9195228          9251407   
2    AR            4801626          4809582          4836980          4882238   
3    AZ           14131942         14190228         14365782         14572440   
4    CA           96550904         96453685         95352868         95271790   

   POPESTIMATE2023  POPESTIMATE2024  NPOPCHG2020  NPOPCHG2021  NPOPCHG2022  \
0          1088238          1094826         -600         2762         1926   
1          9335087          9415717        17601        37780        56179   
2          4926455          4968394         7956        27398        45258   
3         14763422         14982364        58286       175554       206658   
4         95375395         95922475       -97219     -1100817       -81078   

   ...  NETMIG2020  NETMIG2021  NETMIG2022  